## Case Study Questions

1. What is the total amount each customer spent at the restaurant?
2. How many days has each customer visited the restaurant?
3. What was the first item from the menu purchased by each customer?
4. What is the most purchased item on the menu and how many times was it purchased by all customers?
5. Which item was the most popular for each customer?
6. Which item was purchased first by the customer after they became a member?
7. Which item was purchased just before the customer became a member?
8. What is the total items and amount spent for each member before they became a member?
9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

## import data

In [26]:
import polars as pl
import duckdb

db = '~/repos/db-butters/8-week-sql-challenge/case-1/case-1.duckdb'
con = duckdb.connect(db, read_only=True)

sales = con.sql('from dannys_diner.sales;').pl()
menu = con.sql('from dannys_diner.menu;').pl()
members = con.sql('from dannys_diner.members;').pl()

con.close()

### sales table

In [65]:
display(sales.head(10))

customer_id,order_date,product_id
str,date,i32
"""A""",2021-01-01,1
"""A""",2021-01-01,2
"""A""",2021-01-07,2
"""A""",2021-01-10,3
"""A""",2021-01-11,3
"""A""",2021-01-11,3
"""B""",2021-01-01,2
"""B""",2021-01-02,2
"""B""",2021-01-04,1


### menu table

In [30]:
display(menu)

product_id,product_name,price
i32,str,i32
1,"""sushi""",10
2,"""curry""",15
3,"""ramen""",12


### members table

In [31]:
display(members)

customer_id,join_date
str,date
"""A""",2021-01-07
"""B""",2021-01-09


## 1. total spent per customer

In [35]:
(sales.join(menu, on='product_id')
      .group_by('customer_id')
      .agg(
          pl.col('price').sum().alias('total_spent')
      ).sort('customer_id')
)

customer_id,total_spent
str,i32
"""A""",76
"""B""",74
"""C""",36


## 2. customer days at restaurant

In [38]:
(sales.group_by('customer_id')
      .agg(
          pl.col('order_date').n_unique().alias('days_visited')
      ).sort('customer_id')
)

customer_id,days_visited
str,u32
"""A""",4
"""B""",6
"""C""",2


## 3. first item purchased by each customer

In [80]:
(sales.join(menu, on='product_id')
      .group_by(['customer_id', 'order_date'])
      .agg(pl.col('product_id'), pl.col('product_name'))
      .with_columns(pl.col('order_date').rank().over('customer_id').alias('rank'))
      .filter(pl.col('rank') == 1)
      .drop('order_date', 'rank')
      .sort('customer_id')
)

customer_id,product_id,product_name
str,list[i32],list[str]
"""A""","[1, 2]","[""sushi"", ""curry""]"
"""B""",[2],"[""curry""]"
"""C""","[3, 3]","[""ramen"", ""ramen""]"


## 4. most purchased item and number of purchases by each customer

In [83]:
(sales.group_by('product_id')
      .agg(pl.all().len())
)

product_id,customer_id,order_date
i32,u32,u32
3,8,8
2,4,4
1,3,3
